In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import seaborn as sns

# Load combined normalized data
df = pd.read_csv('combined_normalized_data.csv')

print(f"Loaded {len(df)} frames")
print(f"Unique IDs: {df['id'].unique()}")

# ==================== HELPER FUNCTIONS ====================

def get_sequence(df, shot_id):
    """Extract keypoint sequence for a given shot"""
    shot_df = df[df['id'] == shot_id].sort_values('frame_count').reset_index(drop=True)
    kpt_cols = [col for col in df.columns if col.startswith('kpt_')]
    return shot_df[kpt_cols].values

def calculate_angle(p1, p2, p3):
    """Calculate angle at p2 formed by p1-p2-p3"""
    v1 = np.array([p1[0] - p2[0], p1[1] - p2[1]])
    v2 = np.array([p3[0] - p2[0], p3[1] - p2[1]])
    
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    
    if norm1 == 0 or norm2 == 0:
        return 0
    
    cos_angle = np.dot(v1, v2) / (norm1 * norm2)
    angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
    return np.degrees(angle)

def analyze_key_poses(df, shot_id):
    """Analyze critical angles and positions during the smash"""
    shot_df = df[df['id'] == shot_id].sort_values('frame_count').reset_index(drop=True)
    
    analysis = []
    for idx, row in shot_df.iterrows():
        # Key points (assuming right-handed player)
        left_shoulder = (row['kpt_11_x'], row['kpt_11_y'])
        right_shoulder = (row['kpt_12_x'], row['kpt_12_y'])
        left_elbow = (row['kpt_13_x'], row['kpt_13_y'])
        right_elbow = (row['kpt_14_x'], row['kpt_14_y'])
        left_wrist = (row['kpt_15_x'], row['kpt_15_y'])
        right_wrist = (row['kpt_16_x'], row['kpt_16_y'])
        left_hip = (row['kpt_23_x'], row['kpt_23_y'])
        right_hip = (row['kpt_24_x'], row['kpt_24_y'])
        
        # Key angles for right arm (hitting arm)
        right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
        
        # Wrist height relative to shoulder (contact point indicator)
        wrist_height = right_shoulder[1] - right_wrist[1]  # Negative = wrist above shoulder
        
        # Body rotation (hip-shoulder alignment)
        shoulder_width = abs(right_shoulder[0] - left_shoulder[0])
        hip_width = abs(right_hip[0] - left_hip[0])
        
        # Arm extension (distance from shoulder to wrist)
        arm_extension = np.sqrt((right_wrist[0] - right_shoulder[0])**2 + 
                                (right_wrist[1] - right_shoulder[1])**2)
        
        analysis.append({
            'frame': row['frame_count'],
            'elbow_angle': right_elbow_angle,
            'shoulder_angle': right_shoulder_angle,
            'wrist_height': wrist_height,
            'shoulder_width': shoulder_width,
            'hip_width': hip_width,
            'arm_extension': arm_extension
        })
    
    return pd.DataFrame(analysis)

# ==================== DTW ANALYSIS ====================

def compare_motion_sequences(user_seq, reference_seq):
    """Compare two motion sequences using DTW"""
    distance, path = fastdtw(user_seq, reference_seq, dist=euclidean)
    return distance, path

# Extract sequences
user_seq = get_sequence(df, 'user_video')
ref_114_seq = get_sequence(df, 114)
ref_135_seq = get_sequence(df, 135)
ref_148_seq = get_sequence(df, 148)

# Calculate DTW distances
dist_114, path_114 = compare_motion_sequences(user_seq, ref_114_seq)
dist_135, path_135 = compare_motion_sequences(user_seq, ref_135_seq)
dist_148, path_148 = compare_motion_sequences(user_seq, ref_148_seq)

print("\n" + "="*60)
print("DTW DISTANCE ANALYSIS")
print("="*60)
print(f"Distance to Shot 114: {dist_114:.4f}")
print(f"Distance to Shot 135: {dist_135:.4f}")
print(f"Distance to Shot 148: {dist_148:.4f}")

# Use the closest reference
best_ref_id = min([(dist_114, 114), (dist_135, 135), (dist_148, 148)])[1]
print(f"\nBest matching reference: Shot {best_ref_id}")

# ==================== KEY POSE ANALYSIS ====================

user_analysis = analyze_key_poses(df, 'user_video')
ref_114_analysis = analyze_key_poses(df, 114)
ref_135_analysis = analyze_key_poses(df, 135)
ref_148_analysis = analyze_key_poses(df, 148)

# Use the best reference
best_ref_analysis = {114: ref_114_analysis, 135: ref_135_analysis, 148: ref_148_analysis}[best_ref_id]

# Identify impact frame (maximum wrist height - most negative value)
user_impact_idx = user_analysis['wrist_height'].idxmin()
ref_impact_idx = best_ref_analysis['wrist_height'].idxmin()

user_impact = user_analysis.iloc[user_impact_idx]
ref_impact = best_ref_analysis.iloc[ref_impact_idx]

print("\n" + "="*60)
print("IMPACT POINT ANALYSIS")
print("="*60)
print(f"User impact at frame: {user_impact['frame']}")
print(f"Reference impact at frame: {ref_impact['frame']}")

# ==================== FEEDBACK GENERATION ====================

def generate_detailed_feedback(user_analysis, ref_analysis, user_impact, ref_impact, threshold=15):
    """Generate detailed, actionable feedback"""
    feedback = []
    feedback.append("\n" + "="*60)
    feedback.append("BADMINTON SMASH FORM ANALYSIS")
    feedback.append("="*60)
    
    # 1. Elbow Angle Analysis
    elbow_diff = user_impact['elbow_angle'] - ref_impact['elbow_angle']
    feedback.append(f"\n📐 ELBOW ANGLE AT IMPACT:")
    feedback.append(f"   Your angle: {user_impact['elbow_angle']:.1f}°")
    feedback.append(f"   Reference: {ref_impact['elbow_angle']:.1f}°")
    feedback.append(f"   Difference: {elbow_diff:+.1f}°")
    
    if abs(elbow_diff) > threshold:
        if elbow_diff > 0:
            feedback.append(f"   ⚠️  Your elbow is too straight. Bend your elbow more during preparation")
            feedback.append(f"       for explosive power generation.")
        else:
            feedback.append(f"   ⚠️  Your elbow is too bent. Extend more at contact for maximum reach")
            feedback.append(f"       and racket head speed.")
    else:
        feedback.append(f"   ✅ Excellent elbow angle! This is optimal for power generation.")
    
    # 2. Shoulder Angle Analysis
    shoulder_diff = user_impact['shoulder_angle'] - ref_impact['shoulder_angle']
    feedback.append(f"\n💪 SHOULDER ANGLE AT IMPACT:")
    feedback.append(f"   Your angle: {user_impact['shoulder_angle']:.1f}°")
    feedback.append(f"   Reference: {ref_impact['shoulder_angle']:.1f}°")
    feedback.append(f"   Difference: {shoulder_diff:+.1f}°")
    
    if abs(shoulder_diff) > threshold:
        if shoulder_diff > 0:
            feedback.append(f"   ⚠️  Your shoulder is over-rotated. This may cause loss of control.")
        else:
            feedback.append(f"   ⚠️  Not enough shoulder rotation. Rotate your torso more into the shot")
            feedback.append(f"       to generate more power from your core.")
    else:
        feedback.append(f"   ✅ Good shoulder rotation!")
    
    # 3. Contact Point Height
    height_diff = user_impact['wrist_height'] - ref_impact['wrist_height']
    feedback.append(f"\n🎯 CONTACT POINT HEIGHT:")
    feedback.append(f"   Your height: {user_impact['wrist_height']:.3f}")
    feedback.append(f"   Reference: {ref_impact['wrist_height']:.3f}")
    feedback.append(f"   Difference: {height_diff:+.3f}")
    
    if abs(height_diff) > 0.05:  # 5% difference in normalized coordinates
        if height_diff > 0:
            feedback.append(f"   ⚠️  Contact point is too low. Jump higher or time your swing earlier")
            feedback.append(f"       for a steeper downward angle.")
        else:
            feedback.append(f"   ⚠️  Contact point might be too high. Ensure you're hitting at the")
            feedback.append(f"       optimal height for maximum power and control.")
    else:
        feedback.append(f"   ✅ Excellent contact point height!")
    
    # 4. Arm Extension
    extension_diff = user_impact['arm_extension'] - ref_impact['arm_extension']
    feedback.append(f"\n🏹 ARM EXTENSION AT IMPACT:")
    feedback.append(f"   Your extension: {user_impact['arm_extension']:.3f}")
    feedback.append(f"   Reference: {ref_impact['arm_extension']:.3f}")
    feedback.append(f"   Difference: {extension_diff:+.3f}")
    
    if abs(extension_diff) > 0.05:
        if extension_diff < 0:
            feedback.append(f"   ⚠️  Reach higher! Full arm extension maximizes racket head speed.")
        else:
            feedback.append(f"   ⚠️  You're over-extending. Keep some bend for control and prevent injury.")
    else:
        feedback.append(f"   ✅ Good arm extension!")
    
    # 5. Overall Score
    score = 100
    if abs(elbow_diff) > threshold: score -= 20
    if abs(shoulder_diff) > threshold: score -= 20
    if abs(height_diff) > 0.05: score -= 20
    if abs(extension_diff) > 0.05: score -= 15
    
    feedback.append(f"\n⭐ OVERALL FORM SCORE: {score}/100")
    
    if score >= 90:
        feedback.append("   🏆 Excellent form! Your technique matches professional standards.")
    elif score >= 70:
        feedback.append("   👍 Good form with minor adjustments needed.")
    elif score >= 50:
        feedback.append("   📈 Decent form but several areas need improvement.")
    else:
        feedback.append("   💪 Keep practicing! Focus on the key areas mentioned above.")
    
    return "\n".join(feedback)

# Generate and print feedback
feedback = generate_detailed_feedback(user_analysis, best_ref_analysis, user_impact, ref_impact)
print(feedback)

# ==================== VISUALIZATION ====================

def visualize_comparison(user_analysis, ref_analysis, best_ref_id):
    """Create visualization comparing user vs reference"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Form Analysis: User vs Reference Shot {best_ref_id}', fontsize=16, fontweight='bold')
    
    # 1. Elbow Angle over time
    axes[0, 0].plot(user_analysis['frame'], user_analysis['elbow_angle'], 
                    'b-', linewidth=2, label='User', marker='o', markersize=3)
    axes[0, 0].plot(ref_analysis['frame'], ref_analysis['elbow_angle'], 
                    'r--', linewidth=2, label=f'Reference {best_ref_id}', marker='s', markersize=3)
    axes[0, 0].set_xlabel('Frame')
    axes[0, 0].set_ylabel('Elbow Angle (degrees)')
    axes[0, 0].set_title('Elbow Angle Throughout Motion')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Shoulder Angle over time
    axes[0, 1].plot(user_analysis['frame'], user_analysis['shoulder_angle'], 
                    'b-', linewidth=2, label='User', marker='o', markersize=3)
    axes[0, 1].plot(ref_analysis['frame'], ref_analysis['shoulder_angle'], 
                    'r--', linewidth=2, label=f'Reference {best_ref_id}', marker='s', markersize=3)
    axes[0, 1].set_xlabel('Frame')
    axes[0, 1].set_ylabel('Shoulder Angle (degrees)')
    axes[0, 1].set_title('Shoulder Rotation Throughout Motion')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Wrist Height (Contact Point)
    axes[1, 0].plot(user_analysis['frame'], user_analysis['wrist_height'], 
                    'b-', linewidth=2, label='User', marker='o', markersize=3)
    axes[1, 0].plot(ref_analysis['frame'], ref_analysis['wrist_height'], 
                    'r--', linewidth=2, label=f'Reference {best_ref_id}', marker='s', markersize=3)
    axes[1, 0].axhline(y=0, color='gray', linestyle=':', alpha=0.5, label='Shoulder Level')
    axes[1, 0].set_xlabel('Frame')
    axes[1, 0].set_ylabel('Wrist Height (normalized)')
    axes[1, 0].set_title('Contact Point Height Throughout Motion')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Arm Extension
    axes[1, 1].plot(user_analysis['frame'], user_analysis['arm_extension'], 
                    'b-', linewidth=2, label='User', marker='o', markersize=3)
    axes[1, 1].plot(ref_analysis['frame'], ref_analysis['arm_extension'], 
                    'r--', linewidth=2, label=f'Reference {best_ref_id}', marker='s', markersize=3)
    axes[1, 1].set_xlabel('Frame')
    axes[1, 1].set_ylabel('Arm Extension (normalized)')
    axes[1, 1].set_title('Arm Extension Throughout Motion')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('form_analysis_comparison.png', dpi=300, bbox_inches='tight')
    print("\n📊 Visualization saved as 'form_analysis_comparison.png'")
    plt.show()

# Create visualization
visualize_comparison(user_analysis, best_ref_analysis, best_ref_id)

# ==================== SAVE DETAILED REPORT ====================

# Create detailed report CSV
report_data = {
    'Metric': ['DTW Distance', 'Elbow Angle', 'Shoulder Angle', 'Contact Height', 'Arm Extension'],
    'User Value': [
        dist_114,
        user_impact['elbow_angle'],
        user_impact['shoulder_angle'],
        user_impact['wrist_height'],
        user_impact['arm_extension']
    ],
    'Reference Value': [
        0,
        ref_impact['elbow_angle'],
        ref_impact['shoulder_angle'],
        ref_impact['wrist_height'],
        ref_impact['arm_extension']
    ],
    'Difference': [
        dist_114,
        user_impact['elbow_angle'] - ref_impact['elbow_angle'],
        user_impact['shoulder_angle'] - ref_impact['shoulder_angle'],
        user_impact['wrist_height'] - ref_impact['wrist_height'],
        user_impact['arm_extension'] - ref_impact['arm_extension']
    ]
}

report_df = pd.DataFrame(report_data)
report_df.to_csv('form_analysis_report.csv', index=False)
print("\n📄 Detailed report saved as 'form_analysis_report.csv'")

print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)

Loaded 3499 frames
Unique IDs: ['101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111' '112'
 '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123' '124'
 '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135' '136'
 '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147' '148'
 '149' '150' 'user_video']


IndexError: tuple index out of range